[current dir](.)

In [3]:
filename = 'inputs/Day21.txt'
lines = open(filename).read().splitlines()

ipr = int(lines[0].split(' ')[-1])
lines = lines[1:]
print(ipr, len(lines), *lines[:10], sep='\n')

3
31
seti 123 0 5
bani 5 456 5
eqri 5 72 5
addr 5 3 3
seti 0 0 3
seti 0 5 5
bori 5 65536 2
seti 10362650 3 5
bani 2 255 4
addr 5 4 5


In [4]:
import re

inst = []
for line in lines:
    m = re.match('(\w+) (\d+) (\d+) (\d+)', line)
    res = m.groups()
    inst.append([res[0]] + list(map(int, res[1:])))
print(ipr, inst)

3 [['seti', 123, 0, 5], ['bani', 5, 456, 5], ['eqri', 5, 72, 5], ['addr', 5, 3, 3], ['seti', 0, 0, 3], ['seti', 0, 5, 5], ['bori', 5, 65536, 2], ['seti', 10362650, 3, 5], ['bani', 2, 255, 4], ['addr', 5, 4, 5], ['bani', 5, 16777215, 5], ['muli', 5, 65899, 5], ['bani', 5, 16777215, 5], ['gtir', 256, 2, 4], ['addr', 4, 3, 3], ['addi', 3, 1, 3], ['seti', 27, 4, 3], ['seti', 0, 3, 4], ['addi', 4, 1, 1], ['muli', 1, 256, 1], ['gtrr', 1, 2, 1], ['addr', 1, 3, 3], ['addi', 3, 1, 3], ['seti', 25, 2, 3], ['addi', 4, 1, 4], ['seti', 17, 7, 3], ['setr', 4, 0, 2], ['seti', 7, 8, 3], ['eqrr', 5, 0, 4], ['addr', 4, 3, 3], ['seti', 5, 1, 3]]


In [5]:
iset = {
    'addr': lambda registers, a, b: registers[a] + registers[b], 
    'addi': lambda registers, a, b: registers[a] + b,
    'mulr': lambda registers, a, b: registers[a] * registers[b], 
    'muli': lambda registers, a, b: registers[a] * b,
    'banr': lambda registers, a, b: registers[a] & registers[b], 
    'bani': lambda registers, a, b: registers[a] & b,
    'borr': lambda registers, a, b: registers[a] | registers[b], 
    'bori': lambda registers, a, b: registers[a] | b,
    'setr': lambda registers, a, b: registers[a], 
    'seti': lambda registers, a, b: a,
    'gtir': lambda registers, a, b: int(a > registers[b]), 
    'gtri': lambda registers, a, b: int(registers[a] > b), 
    'gtrr': lambda registers, a, b: int(registers[a] > registers[b]),
    'eqir': lambda registers, a, b: int(a == registers[b]), 
    'eqri': lambda registers, a, b: int(registers[a] == b), 
    'eqrr': lambda registers, a, b: int(registers[a] == registers[b])
}

Part 1

In [6]:
def part1():
    ipi, registers = 0, [0] * 6
    for c in range(10000000):
        if ipi >= len(inst):
            break
        registers[ipr] = ipi
        
        # Print registers[5] on instruction 28, since instruction 29 exits if r0==r5 here
        if ipi == 28:
            return registers[5]
            
        l = inst[ipi]
        registers[l[3]] = iset[l[0]](registers, l[1], l[2])
        ipi = registers[ipr] + 1
    
part1()

6778585

Part 2

In [7]:
def part2():
    seen = []
    ipi, registers = 0, [0] * 6
    for c in range(10000000):
        if ipi >= len(inst):
            break
        registers[ipr] = ipi
        
        # Examining the registers reveals that the inner loop translates to
        #    while r1 < r2:
        #        r1 = r4 + 1
        #        r1 *= 256
        #        r4++
        # Therefore, an efficient optimization is, instead of doing the incrementing,
        # to directly set r1 to r2 and calculate r4 back from there.
        # I reverse-engineered the whole thing, but this is sufficient optimization.
        if ipi == 20 and registers[4] == 2:
            registers[1] = registers[2]
            registers[4] = int(registers[1] / 256)-1
            
        # Print the repeating value that could exit if it were r0
        if ipi == 28:
            if registers[5] in seen:
                return seen[-1]
            seen.append(registers[5])
            
        l = inst[ipi]
        registers[l[3]] = iset[l[0]](registers, l[1], l[2])
        ipi = registers[ipr] + 1
    
part2()

6534225